<a href="https://colab.research.google.com/github/cmccomb/example-notebook-workflow/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Additional Inputs for Calculation
# @markdown This is a form that takes some basic inputs
principal_amount = 1000  # @param {type: "number"}
interest_rate = 0.05 # @param {type: "number"}
time_in_years = 3 # @param {type: "number"}

In [ ]:
def perform_calculations(principal_amount, interest_rate, time_in_years):
  """
  Performs some calculations based on the provided inputs, including simple interest.

  Args:
    principal_amount: The principal amount for interest calculation.
    interest_rate: The annual interest rate.
    time_in_years: The time in years for interest calculation.

  Returns:
    The results of the calculation.
  """

  # Simple Interest Calculation
  return principal_amount * interest_rate * time_in_years

In [ ]:
perform_calculations(principal_amount, interest_rate, time_in_years)

150.0